<a href="https://colab.research.google.com/github/RohanSingh7761/Recommendation/blob/main/AllParamsRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [5]:
users_df = pd.read_csv('users_100k_with_relationships.csv')
posts_df = pd.read_csv('posts_100k_with_relationships.csv')
interactions_df = pd.read_csv('interactions_100k_with_relationships.csv')

In [6]:
# Function to extract tags from posts
def extract_tags(post):
    return post['Tags/Keywords'].replace('#', '').split(', ')

# Apply function to posts
posts_df['Tags'] = posts_df.apply(extract_tags, axis=1)
posts_df['Tags_str'] = posts_df['Tags'].apply(lambda tags: ' '.join(tags))

In [7]:
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the TF-IDF Vectorizer on posts tags
tfidf_matrix = tfidf_vectorizer.fit_transform(posts_df['Tags_str'])

In [8]:
# Compute k-nearest neighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [14]:
# Function to get user's friends
def get_user_friends(user_id):
    # Assuming interactions_df contains user relationships
    friends = interactions_df[interactions_df['User ID'] == user_id]['User ID'].unique()
    return friends

In [15]:
 #Function to get posts liked by user's friends
def get_friends_liked_posts(user_id):
    friends = get_user_friends(user_id)
    friends_liked_posts = interactions_df[(interactions_df['User ID'].isin(friends)) & (interactions_df['Interaction Type'] == 'Like')]['Post ID']
    return friends_liked_posts.unique()

In [16]:
# Function to recommend posts based on user's interactions
def recommend_posts(user_id, num_recommended_posts=5):
    # Get user's liked posts
    liked_posts = interactions_df[(interactions_df['User ID'] == user_id) & (interactions_df['Interaction Type'] == 'Like')]['Post ID']

    # Exclude already liked posts
    candidate_posts = posts_df[~posts_df['Post ID'].isin(liked_posts)]

    # Recommend based on k-nearest neighbors with liked posts
    recommended_posts = []
    for post_id in liked_posts:
        post_idx = posts_df.index[posts_df['Post ID'] == post_id].tolist()[0]
        distances, indices = knn.kneighbors(tfidf_matrix[post_idx], n_neighbors=num_recommended_posts+1)
        similar_posts = [(distances[0][i], posts_df['Post ID'].iloc[indices[0][i]]) for i in range(1, num_recommended_posts+1)]
        recommended_posts.extend(similar_posts)

    # Sort and get top recommended posts
    recommended_posts = sorted(recommended_posts, key=lambda x: x[0])

    return [post[1] for post in recommended_posts[:num_recommended_posts]]

In [17]:
# Function to blend posts
def blend_posts(user_id, num_posts=10, friends_ratio=0.2, new_ratio=0.3, interactions_ratio=0.5):
    num_friends_posts = int(num_posts * friends_ratio)
    num_new_posts = int(num_posts * new_ratio)
    num_interactions_posts = num_posts - num_friends_posts - num_new_posts

    # Get posts liked by friends
    friends_liked_posts = get_friends_liked_posts(user_id)
    friends_posts = posts_df[posts_df['Post ID'].isin(friends_liked_posts)]
    if len(friends_posts) < num_friends_posts:
        num_friends_posts = len(friends_posts)
    friends_posts = friends_posts.sample(num_friends_posts)['Post ID'].tolist()

    # Get random new posts
    new_posts = posts_df[~posts_df['Post ID'].isin(friends_liked_posts)]
    if len(new_posts) < num_new_posts:
        num_new_posts = len(new_posts)
    new_posts = new_posts.sample(num_new_posts)['Post ID'].tolist()

    # Get recommended posts based on user interactions
    interaction_posts = recommend_posts(user_id, num_interactions_posts)

    # Blend posts
    blended_posts = friends_posts + new_posts + interaction_posts
    np.random.shuffle(blended_posts)

    return blended_posts

In [20]:
user_id = 100
blended_posts = blend_posts(user_id, num_posts=20)
print(f"Blended posts for User ID {user_id}: {blended_posts}")

Blended posts for User ID 100: [69156, 34172, 67406, 74945, 84435, 16294, 26829, 85587, 35438, 86235, 40472, 91416, 56685, 10447, 77425, 82463, 61789]
